# Resume Screening
## Version 1.0

***
#### Pipeline

1. Resume Parsing  
    * The first subtask is parsing the resume, i.e., extracting information in a structured format from the document. 
2. Resume Matching
    * The second sub-task is extracting semantic information and actually understanding the underlying information.
3. Database Formation

***

### Import Libraries

In [7]:
# !pip install PyMuPDF
# !pip install docx2pdf

In [1]:
import numpy
import sys
import fitz
from docx2pdf import convert

### Reading the file

In [7]:
def read_file(fname):
    if fname.split(".")[1] == 'docx':
        convert(fname)
        fname = fname.split(".")[0] + ".pdf"
    elif fname.split(".")[1] == 'pdf':
        pass
    else:
        print("Only PDF and docx types are supported!")
        return
    
    doc = fitz.open(fname)
    text = ""
    for page in doc:
        text = text + str(page.get_text())

    tx = " ".join(text.split('\n'))  
    print(text)

In [ ]:
read_file('Ahmed Osama-Long Version.doc')

### Pre-processing
* It consists of some main steps
    * Tokenization
    * Stemming
    * Lemmatization
    * Removing Stop Words